In [0]:
!pip install ipywidgets

In [0]:
import pandas as pd
import numpy as np
import xlwings as xw
import ipywidgets as widgets
from IPython.display import display

def donchiain_backtest(stock_id, ndays, start_time, end_time, shares):
    df = pd.read_excel(f"{stock_id}_歷史資料.xlsx")
    df["max20d"] = df["收盤價"].rolling(ndays).max()
    df["min20d"] = df["收盤價"].rolling(ndays).min()
    df["mean20d"] = (df["max20d"] + df["min20d"]) / 2
    df2 = df.set_index("日期")
    df2["max20d"] = df2["max20d"].shift(1)
    df2["min20d"] = df2["min20d"].shift(1)
    df2["mean20d"] = df2["mean20d"].shift(1)
    print(f"start: {start_time}, end: {end_time}")
    df4 = df2[["收盤價", "max20d", "min20d", "mean20d"]][start_time:end_time]
    # 目前是否已做多或做空的訊號
    hold_flag = False
    # 買入股票的訊號
    long_flag = False
    # 做空股票的訊號
    short_flag = False
    # 記錄收益
    balance = 0

    for index, row in df4.iterrows():
        # 若目前沒有進行交易
        if hold_flag == False:
            # 價格低於下限
            if row["收盤價"] <= row["min20d"]:
                # 顯示做空
                df4.loc[index, "交易訊號"] = "Short"
                # 計算總收益
                balance = balance + (row["收盤價"] * shares)
                # 花費現金購買股票
                df4.loc[index, "損益"] = balance
                # 開啓交易訊號
                short_flag = True
                hold_flag = True
                # 價格高於上限
            elif row["收盤價"] >= row["max20d"]:
                # 顯示做多
                df4.loc[index, "交易訊號"] = "Long"
                # 做多，現金減少
                balance = balance - (row["收盤價"] * shares)
                df4.loc[index, "損益"] = balance
                # 開啓交易訊號
                long_flag = True
                hold_flag = True
            else:
                df4.loc[index, "交易訊號"] = "---"
                df4.loc[index, "損益"] = balance
        # 若目前有進行交易
        elif hold_flag == True:
            # 若現在是做多，而且價格大於等於平均值，平倉
            if long_flag == True and row["收盤價"] <= row["mean20d"]:
                df4.loc[index, "交易訊號"] = "Offset"
                # 做多在平倉時，現金增加
                balance = balance + row["收盤價"] * shares
                df4.loc[index, "損益"] = balance
                hold_flag = False
                long_flag = False
                # 平倉之後，將收益歸零
                balance = 0
            # 若現在是做空，而且價格小於等於平均值，平倉
            elif short_flag == True and row["收盤價"] >= row["mean20d"]:
                df4.loc[index, "交易訊號"] = "Offset"
                # 做空在平倉時，現金減少
                balance = balance - row["收盤價"] * shares
                df4.loc[index, "損益"] = balance
                hold_flag = False
                short_flag = False
                # 平倉之後，將收益歸零
                balance = 0
            else:
                df4.loc[index, "交易訊號"] = "---"
                df4.loc[index, "損益"] = balance
    return df4


def on_button_clicked(b):
    # 計算獲勝率及損益
    with output:
        start_time = dp1.value.strftime("%Y-%m-%d")
        end_time = dp2.value.strftime("%Y-%m-%d")
        shares = int(ft1.value)
        ndays = int(ft2.value)
        stock_id = t1.value

        df4 = donchiain_backtest(stock_id, ndays, start_time, end_time, shares)
        result_df = df4[df4["交易訊號"] == "Offset"]
        winrate_B = result_df[result_df["損益"] > 0].shape[0] / result_df.shape[0]
        result_df["纍積損益"] = result_df["損益"].cumsum()
        profit_B = result_df["纍積損益"][-1]
        print(result_df)
        print(f"獲勝率: {winrate_B}")
        print(f"纍積損益: {profit_B}")
        # 輸出至 Excel
        wb = xw.Book()
        sheet = wb.sheets.add(name=f"{stock_id} Donchain 策略B")
        sheet.range("A1").value = result_df
        sheet.range("K1").value = "獲勝率"
        sheet.range("L1").value = winrate_B
        sheet.range("K2").value = "纍計收益"
        sheet.range("L2").value = profit_B
        wb.save(f"{stock_id} Donchian Channel 回測")


dp1 = widgets.DatePicker(
    description='開始日:',
    disabled=False
)

dp2 = widgets.DatePicker(
    description='結束日:',
    disabled=False
)

ft1 = widgets.FloatText(
    value=1000,
    description='交易股數:',
    disabled=False
)

ft2 = widgets.FloatText(
    value=20,
    description='N（天數）:',
    disabled=False
)

t1 = widgets.Text(
    description='股票代號:'
)

display(dp1)
display(dp2)
display(ft1)
display(ft2)
display(t1)
button = widgets.Button(description="開始回測")
output = widgets.Output()
display(button, output)
button.on_click(on_button_clicked)

DatePicker(value=None, description='開始日:')

DatePicker(value=None, description='結束日:')

FloatText(value=1000.0, description='交易股數:')

FloatText(value=20.0, description='N（天數）:')

Text(value='', description='股票代號:')

Button(description='開始回測', style=ButtonStyle())

Output()